So, yesterday when I tried to apply different operations on different columns using a Column Transformer, I got an error stating that the column had missing values. I was confused as my original data didn't have any missing values. So after a lot of trying and testing, I finally figured out that when I was categorizing Age into Age Groups data had 22 values replaced by NaN. I don't know the exact reason why it was happening I did try to find out about it but I couldn't. I proceeded to drop the missing values then and use the column transformer. It ran on the data, though I will proceed further with making pipelines and model fitting but I assume it will not give a good result since I didn't handle the error correctly. But I will move forward and try to find the answer while learning and practicing. Maybe with experience, I would easily find out errors. 

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
df = pd.read_csv("Bank_Churn.csv")
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [36]:
df['AgeGroup'] = pd.cut(df['Age'], bins=[18, 25, 35, 45, 60,100], labels=['18-25', '26-35', '36-45', '46-60', '60+'])
df['CreditScoreCat'] = pd.cut(df['CreditScore'], 
                                   bins=[300, 580, 669, 739, 799, 850], 
                                   labels=['Poor', 'Fair', 'Good', 'Very Good', 'Exceptional'])


In [37]:
new_df = df.drop(columns = ['CustomerId','Surname','CreditScore','Age'])
new_df 

,Geography,Gender,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,AgeGroup,CreditScoreCat
0,France,Female,2,0.00,1,1,1,101348.88,1,36-45,Fair
1,Spain,Female,1,83807.86,1,0,1,112542.58,0,36-45,Fair
2,France,Female,8,159660.80,3,1,0,113931.57,1,36-45,Poor
3,France,Female,1,0.00,2,0,0,93826.63,0,36-45,Good
4,Spain,Female,2,125510.82,1,1,1,79084.10,0,36-45,Exceptional
...,...,...,...,...,...,...,...,...,...,...,...
9995,France,Male,5,0.00,2,1,0,96270.64,0,36-45,Very Good
9996,France,Male,10,57369.61,1,1,1,101699.77,0,26-35,Poor
9997,France,Female,7,0.00,1,0,1,42085.58,1,36-45,Good
9998,Germany,Male,3,75075.31,2,1,0,92888.52,1,36-45,Very Good


In [38]:
new_df.isnull().sum()

Geography           0
Gender              0
Tenure              0
Balance             0
NumOfProducts       0
HasCrCard           0
IsActiveMember      0
EstimatedSalary     0
Exited              0
AgeGroup           22
CreditScoreCat      0
dtype: int64

In [31]:
null_age_rows = new_df[new_df['AgeGroup'].isnull()]
print(null_age_rows)

     Geography  Gender  Tenure    Balance  NumOfProducts  HasCrCard  \
746     France    Male       2  160980.03              1          0   
1619    France    Male      10  151762.74              1          0   
1678    France  Female       4   82767.42              1          1   
2021   Germany  Female       3  128743.80              1          0   
2136    France    Male       4  133550.67              1          1   
2141    France    Male       7       0.00              2          1   
3330    France    Male       4       0.00              2          1   
3512     Spain    Male       3       0.00              2          1   
3517     Spain    Male       1       0.00              2          0   
3686    France    Male       2  176139.50              2          1   
4556    France  Female       1   98894.39              1          1   
4716    France    Male      10       0.00              2          0   
7334    France    Male       6       0.00              2          1   
7722  

In [42]:
new_df = new_df.dropna() 

In [43]:
new_df.shape

(9978, 11)

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(new_df.drop(columns=['Exited']) ,new_df['Exited'],test_size = 0.3, random_state = 0)
X_train.shape, X_test.shape

((6984, 10), (2994, 10))

In [45]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [46]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',StandardScaler(),['Balance','EstimatedSalary']),
    ('tnf2',OrdinalEncoder(categories=[['Poor', 'Fair', 'Good', 'Very Good', 'Exceptional']]),['CreditScoreCat']),
    ('tnf3',OrdinalEncoder(categories=[['18-25', '26-35', '36-45', '46-60', '60+']]),['AgeGroup']),
    ('tnf4',OneHotEncoder(sparse_output = False, drop = 'first'),['Gender','Geography'])
],remainder = 'passthrough')

In [47]:
transformer.fit_transform(X_train)

array([[ 1.44322276,  0.85497264,  2.        , ...,  1.        ,
         1.        ,  1.        ],
       [-1.22447479,  1.25306411,  3.        , ...,  2.        ,
         1.        ,  1.        ],
       [ 1.19483406,  0.61803897,  0.        , ...,  2.        ,
         1.        ,  1.        ],
       ...,
       [ 0.66896644,  1.457473  ,  4.        , ...,  1.        ,
         1.        ,  0.        ],
       [-1.22447479,  1.39387145,  1.        , ...,  2.        ,
         0.        ,  0.        ],
       [ 0.84232863,  0.23371777,  1.        , ...,  3.        ,
         0.        ,  0.        ]])

In [48]:
transformer.fit_transform(X_train).shape

(6984, 11)

In [49]:
transformer.transform(X_test).shape

(2994, 11)

This is my first time using a Column Transformer. I can't figure out why the transfrom fit is throwing an error of Nan values when there are not. I will identify it soon and update here. Okay so I identified that when I changed Age to Age Group That is divided the age in categories my data had encounted 22 missing values that were not in the data initially. So for now i am dropping the missing values and proceed with the column transformer.

I am trying to use pipelines. Right now I cannot come to conclusion why am I getting this error.

In [25]:
import pandas as pd

data = {'Name': ['John', 'Mary', None, 'Peter'], 'Age': [20, 25, 30, None]}
df = pd.DataFrame(data)

# # To see rows with any null values:
# null_rows = df[df.isnull().any(axis=1)]
# print(null_rows)

# To see rows with null values in a specific column:
null_age_rows = df[df['Age'].isnull()]
print(null_age_rows)

    Name  Age
3  Peter  NaN
